In [1]:
# Convolutional neural network from scratch 
import cv2
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('mnist_train.csv')
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = np.array(data) # transform our csv into array 
lines, cols = data.shape # lines = 42000 / columns = 785 bcz the images are 28x28=784 pixels +1 with the beggining cols of label
np.random.shuffle(data) # because the train data are possibly sorted with a sequences of 0 then 1 then 2 and so on 
# we shuffe the dataset in order to avoid overfitting
data_validation = data[0:1000].T # Data transposition is performed in this particular context to facilitate 
# the manipulation of examples and features. Some machine learning algorithms and libraries prefer 
# this matrix representation with examples in columns rather than rows.
# data validation variable is used to evaluate and confirm the performance of our future model while training
label_val = data_validation[0]
result_val = data_validation[1:cols]
result_val = result_val / 255. # we normalise between 0 and 1 in order to facilitate the process of learning
# The dot is here to indicate that the division should be done with a decimal precision

data_train = data[1000:lines].T
label_train = data_train[0] # we did the transpose so the first line is compose only by label
result_train = data_train[1:cols]
result_train = result_train / 255.

In [10]:
def init_params() :
    # W => weights / b => bias ... In the beginning both are generated randomly 
    # 1st param : nb neurons on the actual layer / 2st param : nb neurons of the previous layer
    neuronsLayerOne = 10 
    neuronsLayerTwo = 10
    W1 = np.random.rand(neuronsLayerOne,cols-1) -0.5 # -1 => to understand refer to what we said on the line of creation of cols
    b1 = np.random.rand(neuronsLayerOne,1) - 0.5 # which means for each neurons we generate a random number which will corresponding to the bias
    # bias doesn't have to be necessary between 0 and 1 ! 
    W2 = np.random.rand(neuronsLayerTwo,neuronsLayerOne) -0.5
    b2 = np.random.rand(neuronsLayerTwo,1) -0.5
    # - 0.5 is in reality not necessary but bcz these params are gonna be use in softmax function 
    # which include exponential if we've got a too big number, the exponential will be too hard to compute for the computer 
    # In other words - 0.5 is only here due to computer's performance 
    return W1, b1, W2, b2

def ReLu(Z) : #activation function non linear 
    return np.maximum(Z,0)

def softmax(Z) : 
    return np.exp(Z)/sum(np.exp(Z))

def deriv_ReLu(Z) :
    return Z>0

def forward_propagation(W1, b1, W2, b2, X) : # X is the input data (pixels 1 image) / Z -> unactivated layer / A -> activated 
    # forward propagation is when you take images and run them through the CNN / It's a linear transformation
    Z1 = W1.dot(X)+b1 # X -> several lines and 1 column due to the transpose ! (l,c) x (l,c)  W1 -> l : 1 c : 10 
    # we tend to represent layer of CNN in line but we do not need to adapt this in our line of code bcz it's not important
    A1 = ReLu(Z1)
    Z2 = W2.dot(A1)+b2
    A2 = softmax(Z2) # final result 
    # final result Y of our CNN will be score associated to each prediction such as this
    # [0 0.05 0.7 0.01 0.09 0.1 0 0.05 0 0] = total equals to 1 such as a distribution function !
    'RESUME : 
    'Forward propagation for image is taking all the pixels of the image and applying them
    'some operations : 
    'SUM (Entry pixels X Weight + bias)
    'Non linearity => activation function (ReLu and softmax here)
    return Z1, A1, Z2, A2

def one_hot(Y) : # function that will take the label of our data and for each label transform it into an encoded matrix 
    # example : 0 : 100000000 / 4 : 0000100000 / 9 : 0000000001 ...
    labelsNb = 10 # 0 1 2 3 4 5 6 7 8 9
    one_hot_Y = np.zeros((Y.size, labelsNb)) # to specify tuple we put () / Y.size => number of values of labels that we pass to the function 
    one_hot_Y[np.arange(Y.size),Y] = 1 
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_propagation(Z1, A1, Z2, A2, W2, X, Y) : 
    m = Y.size # equals line
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y # value of the error 
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2) # average of the absolute error 
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1) #TO define 
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    'Documentation'
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha): 
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2  
    'Documentation'
    return W1, b1, W2, b2

In [11]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_propagation(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [12]:
W1, b1, W2, b2 = gradient_descent(label_train,result_train,  0.10, 500)

ValueError: shapes (10,784) and (59000,) not aligned: 784 (dim 1) != 59000 (dim 0)

In [38]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.

In [43]:
def init_params():
    W1 = np.random.rand(10, 784) -0.5
    b1 = np.random.rand(10, 1)  -0.5
    W2 = np.random.rand(10, 10) -0.5
    b2 = np.random.rand(10, 1) -0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [44]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [45]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[1 1 1 ... 5 1 8] [3 3 5 ... 0 2 4]
0.11067796610169492
Iteration:  10
[1 1 1 ... 5 1 8] [3 3 5 ... 0 2 4]
0.1477627118644068
Iteration:  20
[1 2 1 ... 5 1 8] [3 3 5 ... 0 2 4]
0.1709322033898305
Iteration:  30
[1 8 9 ... 5 1 8] [3 3 5 ... 0 2 4]
0.1951864406779661
Iteration:  40
[1 8 5 ... 5 1 8] [3 3 5 ... 0 2 4]
0.21894915254237288
Iteration:  50
[6 8 5 ... 5 1 8] [3 3 5 ... 0 2 4]
0.23535593220338982
Iteration:  60
[6 1 5 ... 5 1 8] [3 3 5 ... 0 2 4]
0.25245762711864406
Iteration:  70
[6 1 5 ... 5 1 0] [3 3 5 ... 0 2 4]
0.27208474576271185
Iteration:  80
[6 1 5 ... 5 1 0] [3 3 5 ... 0 2 4]
0.2931186440677966
Iteration:  90
[3 3 5 ... 5 1 0] [3 3 5 ... 0 2 4]
0.32266101694915256
Iteration:  100
[3 3 5 ... 5 1 0] [3 3 5 ... 0 2 4]
0.3717966101694915
Iteration:  110
[3 3 5 ... 5 1 0] [3 3 5 ... 0 2 4]
0.40147457627118643
Iteration:  120
[3 3 5 ... 5 1 0] [3 3 5 ... 0 2 4]
0.4302881355932203
Iteration:  130
[3 3 5 ... 5 1 0] [3 3 5 ... 0 2 4]
0.46547457627118644
Iteration